In [2]:
import sys, os, time, shutil, random
from pathlib import Path
_cwd = os.getcwd()
os.chdir(Path(_cwd)/'..')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# %load_ext autoreload
# %autoreload 2
# %pdb
from itertools import product, cycle
from heapq import heapify, heappush, heappop
import cv2
import omnifig as fig
import numpy as np
np.set_printoptions(linewidth=120)
import pickle
from tabulate import tabulate
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
# import torchvision.models
from torch.utils.data import Dataset, DataLoader, TensorDataset
import networkx as nx

%matplotlib notebook
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.cm
from matplotlib import animation
import matplotlib as mpl
from matplotlib.collections import PatchCollection
# mpl.rc('image', cmap='gray')
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import normalize

fig.initialize()

import omnilearn as learn
from omnilearn import models
from omnilearn import util
from omnilearn.data import InterventionSamplerBase
from omnilearn import viz as viz_util

from sklearn.cluster import MeanShift, estimate_bandwidth, DBSCAN
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA

from src import sample_full_interventions, response_mat
from src.responses import sample_full_interventions

# import gpumap
from c3linearize import linearize, class_graph


In [3]:
device = 'cuda'
# device = 'cuda:1'
dataset = None
root = Path(r'C:/Users/anwan/Documents/workspace/rfd_models/')
figure_root = Path('figures/responses')
figure_root = None
if figure_root is not None and not figure_root.exists():
    figure_root.mkdir()
seed = 10
B = 128
N = 16
G = 32

In [9]:
run_names = '''rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64_210524-161408
rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-wdp0001_210524-161607
rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708
rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908
rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205
rfd_vae2_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214633
rfd_vae1_e-l16_d-l16_d16-strd-trans-lrp00005-bs64-ch256_210524-214809
rfd_wae10_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-215009
rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527'''.split('\n')
run_names = run_names[2:]
print(tabulate(enumerate(run_names)))

-  ---------------------------------------------------------------------------
0  rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708
1  rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908
2  rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205
3  rfd_vae2_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214633
4  rfd_vae1_e-l16_d-l16_d16-strd-trans-lrp00005-bs64-ch256_210524-214809
5  rfd_wae10_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-215009
6  rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527
-  ---------------------------------------------------------------------------


In [5]:
# print('\n'.join([f'unbuffer fig eval-fix-hybrid eval/rfd-fid --load {r} --mode test --ident final' for r in run_names]))

In [10]:
runs = [fig.quick_run('load-run', path=row, root=str(root), **{'override.device':'cpu'}) for row in run_names]

ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"
ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"
ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"


| path: 'rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708'
| saveroot: None (by default)
| root: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models'
| [Pushed] path: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708'
| run: [<class 'omnifig.config.ConfigDict'>]
| CREATING . (type=sae-run) (mod={'cls-run': 3, 'inline': 5, 'timed-run': 1})
|  > silent: None (by default)
|  > invisible: False (by default)
|  > use_config_root: True (by default)
|  > .path: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708'
|  > [Pushed] path: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708'
|  > ckpt-num: None (by default)
|  > best: False (by default)
|  > last: True (by default)
|  > [Pushed] dataset._load-ckpt: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_ae_e

ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"
ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"
ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"


|  > silent: None (by default)
|  > invisible: False (by default)
|  > use_config_root: True (by default)
|  > .path: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205'
|  > [Pushed] path: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205'
|  > ckpt-num: None (by default)
|  > best: False (by default)
|  > last: True (by default)
|  > [Pushed] dataset._load-ckpt: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205\\ckpt200000'
|  > [Pushed] model._load-ckpt: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205\\ckpt200000'
|  > [Pushed] records._load-ckpt: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205\\ckpt200000'
|  > [Pushed] 

ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"


| saveroot: None (by default)
| root: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models'
| [Pushed] path: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527'
| run: [<class 'omnifig.config.ConfigDict'>]
| CREATING . (type=sae-run) (mod={'cls-run': 3, 'inline': 5, 'timed-run': 1})
|  > silent: None (by default)
|  > invisible: False (by default)
|  > use_config_root: True (by default)
|  > .path: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527'
|  > [Pushed] path: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527'
|  > ckpt-num: None (by default)
|  > best: False (by default)
|  > last: True (by default)
|  > [Pushed] dataset._load-ckpt: 'C:\\Users\\anwan\\Documents\\workspace\\rfd_models\\rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_

In [21]:
titles = ['SAE-16', 'AE', 'VAE', r'$\beta$VAE', 'LVAE-16', 'WAE', 'AdaAE-16']
archs = ['Structural', 'Baseline', 'Baseline', 'Baseline', 'Ladder', 'Baseline', 'Adaptive']
cs = ['C0', 'C1', 'C1', 'C1', 'C3','C1',  'C2']
cut=  'eval'

In [22]:
for n, a, c, r in zip(titles, archs, cs, runs):
    r.title = n
    r.arch = a
    r.color = c
    out = r.get_results(cut)
    r.rec_fid = out['rec_fid']
    r.hyb_fid = out['hybrid_fid']
    r.prior_fid = out.get('prior_fid', None)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ordered = sorted(runs, key=lambda r: -r.rec_fid)
ac = set()
for r in runs:
    ac.append()

In [ ]:
# old

fig, ax = plt.subplots(figsize=(5,5))
# texts = []
tps = set()
x = np.arange(len(title_order))

order = [x[1] for x in sorted((r.stats.rec_fid, r.ident) for r in runs)]
for i,t in enumerate(order):
    r = run_idents[t]
    s = r.stats.rec_fid
    c = info.colors[r.group]
    plt.bar(i, s, color=c, edgecolor='k', )#linewidth=2)
    tps.add(r.group)
plt.xticks(x, [titles[t] for t in order], rotation='vertical')
#     plt.scatter(r.loss, r.fid, color=colors[r.type])
#     texts.append(plt.text(r.loss, r.fid, r.name))
#     tps.add(r.type)
#     plt.scatter(r.fid, r.loss, color=colors[r.type])
#     texts.append(plt.text(r.fid-(fidrg/10 if r.type == 'attn' else 0), r.loss, r.name))
# legend_elements = [Patch(facecolor=info.colors[name], label=info.tnames[name]) for name in set(typs)]
legend_elements = [Patch(facecolor=info.colors[name], label=info.group_names[name]) for name in tps]
legend = plt.legend(handles=legend_elements, title='Architecture')
# plt.ylabel('Reconstruction Loss')
plt.ylabel('Reconstruction FID Score')
plt.title(f'{info.title} Reconstruction Quality')
# plt.yscale('log')
# s = [20,30,40,50,60,70, 80,90,100]
if dscode == 'celeb':
    plt.ylim(80,150)
elif dscode in {'3ds', 'toy'}:
    plt.ylim(0,50)
elif dscode == 'real':
    plt.ylim(0,70)
# else:
#     plt.ylim(0,100)
# plt.ylim(4550,4580)
# plt.yticks(s,map(str,s))
plt.grid(linestyle='--',axis='y')

# legend1 = plt.legend(legend_elements1, ['Hybrid', 'Prior'], loc=4, title='Sampling Method')
# ax.add_artist(legend1, )
# ax.add_artist(legend)
plt.tight_layout()
util.save_figure(f'{dscode}_rec-fid', **save_figure_kwargs)
pass

In [19]:
for r in runs:

In [15]:
out = r.get_results('eval')
out.keys()

dict_keys(['device', 'rec_fid_stats', 'rec_fid', 'hybrid_fid_stats', 'hybrid_fid', 'batch'])

In [16]:
out['rec_fid']

6.6508499285524465

In [6]:
run_name = random.choice(run_names)

run_name = run_names[2]
run_name = run_names[3]

run_name

'rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908'

In [5]:
load_config = fig.get_config(path=run_name, root=str(root), **{'override.device':device})
load_config.set_silent(True)
run = fig.run('load-run', load_config)
A = run.get_config()
A.push('model-override.model._model_mod.hybrid', 1)
A.set_silent(True)
info = A.pull('info')
info

ERROR:omnifig.organization: Could not find mod "cls-run" for component "sae-run"


{'arch_type': 'c16',
 'dataset_type': 'rfd',
 'dec_type': 'c16',
 'enc_type': 'c16',
 'extra': 'd16-strd-trans-lrp00005-bs64-ch256',
 'model_type': 'ae',
 'size': 128}

In [6]:
model = run.get_model()
model.switch_to('val')
model.to(device);

Hybrid_Autoencoder(
  (encoder): MultiLayer(
    (layers): ModuleList(
      (0): ConvLayer(
        residual=False
        (conv): Conv2d(3, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        (norm): GroupNorm(8, 256, eps=1e-05, affine=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (1): ConvLayer(
        residual=False
        (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): GroupNorm(8, 256, eps=1e-05, affine=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (2): ConvLayer(
        residual=False
        (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): GroupNorm(8, 256, eps=1e-05, affine=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (3): ConvLayer(
        residual=False
        (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): GroupNorm(8, 256, eps=1e-05, affine=True)
        (nonlin): 

Loaded parameters from C:\Users\anwan\Documents\workspace\rfd_models\rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908\ckpt200000


In [7]:
# data = run.get_results('eval')
# data.keys()

In [8]:
# data['hybrid_fid']

In [9]:
if dataset is None:
    dataset = run.get_dataset()
len(dataset)

====== Opening RFD Dataset ======


720000

In [10]:
dataset.get_available_modes()

{'heldout_test', 'real_test', 'test', 'train'}

In [11]:
batch = dataset.get_batch(batch_size=128, shuffle=True)
batch.shape

torch.Size([128, 3, 128, 128])

In [12]:
with torch.no_grad():
    Q = model.encode(batch)
    if isinstance(Q, distrib.Normal):
        Q = Q.loc
    if model._latent is None:
        model._latent = Q
Q.shape

torch.Size([128, 16])

In [13]:
A.update({
    'novel': True,
    'skip-expensive': True,
    'dataset.batch_size': 16,
    'evaluation': None,
    'inline': True,
    'pbar._type': 'progress-bar',
    'pbar.display-on': 'jupyter',
    'fid.dim': 2048,
    'fid.n_samples': 50000,
    'fid.batch_size': 32,
})

In [ ]:
out = run.evaluate(overwrite=True)

Will save results to C:\Users\anwan\Documents\workspace\rfd_models\rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908
Loading inception model dim=2048


  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
out.keys()

In [25]:
{k:v for k,v in out.items() if k.endswith('_fid')}

{'rec_fid': 16.907577234205405,
 'prior_fid': 35.69107753449731,
 'hybrid_fid': 28.269544532424334}

In [25]:
{k:v for k,v in out.items() if k.endswith('_fid')}

{'rec_fid': 16.907577234205405,
 'prior_fid': 35.69107753449731,
 'hybrid_fid': 28.269544532424334}

In [15]:
{k:v for k,v in out.items() if k.endswith('_fid')}

{'rec_fid': 9.521835270454687, 'hybrid_fid': 19.730062040454442}